In [1]:
API_KEY = 'xxx'
PROJECT_ID = 'xxx'

In [6]:
import os
import json
from PIL import Image


# get image size
def get_image_size(image_path):
    with Image.open(image_path) as img:
        return img.size  # (width, height)

# the fuction of YOLO foramt convert to Labelbox format
def yolo_to_labelbox(yolo_label, img_width, img_height):
    parts = yolo_label.split()
    category = int(parts[0])
    center_x_ratio = float(parts[1])
    center_y_ratio = float(parts[2])
    width_ratio = float(parts[3])
    height_ratio = float(parts[4])
    
    center_x = center_x_ratio * img_width
    center_y = center_y_ratio * img_height
    width = width_ratio * img_width
    height = height_ratio * img_height
    
    top = max(0, min(top, img_height))
    left = max(0, min(left, img_width))
    bottom = max(0, min(bottom, img_height))
    right = max(0, min(right, img_width))

    top = center_y - (height / 2)
    left = center_x - (width / 2)
    bottom = center_y + (height / 2)
    right = center_x + (width / 2)
    
    labelbox_annotation = {
        "name": "Vehicle Class Ontology",
        "value": {
            "start": {"x": left, "y": top},
            "end": {"x": right, "y": bottom}
        },
        "classifications": [{
            "name": "Vehicle Type",
            "value": {
                "answer": {
                    "name": "car"
                }
            }
        }]
    }
    
    return labelbox_annotation

In [7]:
# convert txt file to json file
def process_folder(label_folder, image_folder, output_folder):
    result = {}
    for file in os.listdir(label_folder):
        if file.endswith(".txt"):
            image_path = os.path.join(image_folder, file.replace('.txt', '.jpg'))  
            
            if not os.path.exists(image_path):
                print(f"Warning: Image {image_path} not found. Skipping corresponding label {file}.")
                continue
            
            with open(os.path.join(label_folder, file), 'r') as f:
                labels = f.readlines()
            

            img_width, img_height = get_image_size(image_path)
       
            converted_labels = [yolo_to_labelbox(label.strip(), img_width, img_height) for label in labels]
         
            result[file.replace('.txt', '')] = converted_labels
    
            if not os.path.exists(output_folder):
                os.makedirs(output_folder)

            output_file_path = os.path.join(output_folder, file.replace('.txt', '.json'))
            with open(output_file_path, 'w') as f:
                json.dump(converted_labels, f, indent=4)

process_folder(r'labels folder', r'image folder', r'converted label folder')

In [2]:
OUTPUT_DIRECTORY_PATH = r'converted label folder'

In [ ]:
import os
import requests

# get mapping between externalId and uid

file_names = [f[:-5] for f in os.listdir(OUTPUT_DIRECTORY_PATH) if f.endswith('.json')]
external_ids = [f"{name}.jpg" for name in file_names]

headers = {
    'Authorization': f'Bearer {API_KEY}',
    'Content-Type': 'application/json'
}

query = """
query GetDataRowID($projectId: ID!, $externalId: String!) {
  project(where: {id: $projectId}) {
    dataRows(where: {externalId: $externalId}) {
      id
    }
  }
}
"""
data_row_ids = []
id_mapping = {}

for external_id in external_ids:
    variables = {
        'projectId': PROJECT_ID,
        'externalId': external_id
    }

    response = requests.post(
        'https://api.labelbox.com/graphql',
        headers=headers,
        json={'query': query, 'variables': variables}
    )

    data = response.json()
    data_row_id = data['data']['project']['dataRows'][0]['id']
    data_row_ids.append(data_row_id)
    id_mapping[external_id] = data_row_id

print(id_mapping)

In [10]:
import os
import json
import uuid
import labelbox as lb
import labelbox.data.annotation_types as lb_types
from labelbox.data.annotation_types import ClassificationAnnotation

client = lb.Client(API_KEY)
project = client.get_project(PROJECT_ID)

def convert_to_labelbox_format(label_folder):
    all_annotations = []
    for file in os.listdir(label_folder):
        if file.endswith(".json"):
            with open(os.path.join(label_folder, file), 'r') as f:
                annotations_data = json.load(f)

            annotations = []
            for annotation in annotations_data:
                bbox = lb_types.ObjectAnnotation(
                    name=annotation['name'],
                    value=lb_types.Rectangle(
                        start=lb_types.Point(x=annotation['value']['start']['x'], y=annotation['value']['start']['y']),
                        end=lb_types.Point(x=annotation['value']['end']['x'], y=annotation['value']['end']['y'])
                    ),
                    classifications=[
                        lb_types.ClassificationAnnotation(
                            name=annotation['classifications'][0]['name'],
                            value=lb_types.Radio(
                                answer=lb_types.ClassificationAnswer(name=annotation['classifications'][0]['value']['answer']['name'])
                        )
                        )
                    ]

                )
                annotations.append(bbox)
                
            external_id = file.replace('.json', '.jpg')
            
            uid_value = id_mapping.get(external_id)

            image_data = lb_types.ImageData(uid = uid_value)
            
            label_data = lb_types.Label(data=image_data, annotations=annotations)
            
            all_annotations.append(label_data)
    
    return all_annotations

labels = convert_to_labelbox_format(r'converted label folder')

print(labels)

[Label(uid=None, data=ImageData(im_bytes=None,file_path=None,url=None,arr=None), annotations=[ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=78.9999408, y=45.99992400000001), end=Point(extra={}, x=92.9999312, y=59.999916)), classifications=[ClassificationAnnotation(confidence=None, name='Vehicle Type', feature_schema_id=None, extra={}, value=Radio(confidence=None, answer=ClassificationAnswer(confidence=None, name='pickup_truck', feature_schema_id=None, extra={}, keyframe=None, classifications=[])), message_id=None)]), ObjectAnnotation(confidence=None, name='Vehicle Class Ontology', feature_schema_id=None, extra={}, value=Rectangle(extra={}, start=Point(extra={}, x=78.9999408, y=45.99992400000001), end=Point(extra={}, x=92.9999312, y=59.999916)), classifications=[ClassificationAnnotation(confidence=None, name='Vehicle Type', feature_schema_id=None, extra={}, value=Radio(confidence=None,

In [11]:
upload_job_label_import = lb.LabelImport.create_from_objects(
    client=client,
    project_id=project.uid,
    name="label_import_job_success18",
    labels=labels
)
print("Errors:", upload_job_label_import.errors)

Errors: [{'uuid': '15323a73-8b96-4182-b94e-806f4b628769', 'dataRow': {'id': 'clo4wb7n100bo0708c9f7eney', 'globalKey': None}, 'status': 'FAILURE', 'errors': [{'name': 'InvalidImageAnnotation', 'message': "The point (x: 57.99992, y: 240.000084, lies outside of the asset's dimensions (352x240)", 'additionalInfo': None}]}, {'uuid': '2d09a0bf-c10a-45ba-af4a-bb124f63dbc2', 'dataRow': {'id': 'clo4wb7n100c007088bju7vqi', 'globalKey': None}, 'status': 'FAILURE', 'errors': [{'name': 'InvalidImageAnnotation', 'message': "The point (x: 352.00007, y: 136.00008, lies outside of the asset's dimensions (352x240)", 'additionalInfo': None}]}, {'uuid': 'b9bb2ce1-c636-401f-a285-449cbff70a43', 'dataRow': {'id': 'clo4wb7n100c707081ao76iie', 'globalKey': None}, 'status': 'FAILURE', 'errors': [{'name': 'InvalidImageAnnotation', 'message': "The point (x: 2.000064, y: 240.00012, lies outside of the asset's dimensions (352x240)", 'additionalInfo': None}]}, {'uuid': 'c231adef-0703-42f3-938b-55810964a884', 'dataRo